## 1 - Objetivo

A intenção deste exercício é observar as diferenças de aprendizado entre os algoritmos e como os resultados podem variar entre eles. 

Neste primeiro momento nós vamos utilizar funções padrão da biblioteca scikit-learn para estudar os algoritmos básicos de machine learning. Como nossa atividade é bastante simples vamos utilizar datasets que vem pré-construídos na biblioteca.



## 2 - Carregando as bibliotecas

Scikit-learn possui uma interface limpa e intuitiva e todos os componentes da biblioteca expõem a mesma interface de métodos. Sendo uma biblioteca fácil de usar, tornou-se padrão na indústria de tecnologia.

Para utilizá-la, vamos primeiro carregar os métodos/módulos necessários

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt  # primeiro importamos a biblioteca para visualização
import numpy as np  # importamos também a biblioteca NumPy que irá nos fornecer diversos métodos para trabalhar com arrays
import seaborn as sns
import pandas as pd

Antes de iniciarmos o nosso problema, nós também devemos definir algumas funções que nos auxiliarão no tutorial.

In [ ]:
def plot_iris(X):
    """ Função para visualização do dataset iris. Embora seja um dataset 
    com 4 features, vamos utilizar apenas as 2 primeiras para gerar um 
    gráfico
    """
    plt.figure(2, figsize=(8, 6))
    plt.clf()
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1,
            edgecolor='k')
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xticks(())
    plt.yticks(())

from matplotlib.colors import ListedColormap
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
from sklearn.metrics import accuracy_score

def plot_resultado(X_, y_, clf, title):
    X = X_[:, :2]
    y = y_

    h = .02  # step size in the mesh
    # we create an instance of Neighbours Classifier and fit the data.
    clf.fit(X, y)

    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    
    preds = clf.predict(X)
    acc = accuracy_score(y, preds)
    print("Percentual de acertos {:.2f}".format(acc))

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure()
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold,
                edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title(title)

#### 2.1 - Dataset

Nós vamos utilizar o *Iris flower dataset* (ou *Fisher's Iris dataset*), criado pelo estatístico e biólogo britânico Ronald Fisher em seu artigo "*The use of multiple measurements in taxonomic problems*", de 1936, no qual apresentou a primeira versão da análise de discriminantes lineares.

Este dataset contém 50 exemplos de três espécies de flores de Íris (*Iris setosa*, *Iris virginica* e *Iris versicolor*). Cada datapoint consiste de 4 features: comprimento e largura da sépala, comprimento e largura das pétalas (todos em centímetros).

Este dataset se tornou um caso de teste típico para algoritmos de machine learning com foco em classificação.


In [ ]:
from sklearn import datasets # aqui importamos o módulo de datasets

# aqui nós carregamos o dataset
iris = datasets.load_iris() 

# nós vamos dividir nosso dataset em X (features) e y (alvo)
X = iris.data 
y = iris.target

# vamos agora dividir o dataset em treino e teste
indices = np.random.permutation(len(X))  # primeiro obtemos uma lista de ordem aleatória dos índices do dataset
X_train = X[indices[:-10]]  # obtemos uma parte das features para treino
y_train = y[indices[:-10]]  # obtemos a parte correspondente dos alvos para treino
X_test  = X[indices[-10:]]  # obtemos uma parte das features para teste
y_test  = y[indices[-10:]]  # obtemos a parte correspondente dos alvos para teste

# criando uma visualização básica
sns.pairplot(pd.read_csv("data/iris.csv"), hue="species")

Como podemos ver acima, nosso problema possui 3 classes a serem classificadas. Uma delas aparenta ser simples, ao passo que as outras estão um pouco embaralhada!

*Observação*: devido ao fato de conseguirmos criar um gráfico em 2 dimensões, nós vamos utilizar apenas as 2 primeiras features neste exercício. É importante lembrar que este exercício tem a intenção de nos dar algumas intuições sobre o machine learning e o funcionamento de seus algoritmos e não de ser uma representação exata de suas capacidades.

## 3 - Os modelos

O scikit-learn é uma biblioteca que implementa (quase) todos os principais algoritmos usados em machine learning, assim como diversos métodos utilitários para feature engineering e feature selection.

O principal método que todo o algoritmo implementa é o  ```fit```. Desta forma, fica fácil criarmos/substituirmos um algoritmo:

```
algoritmo.fit(features, alvos)
```

O scikit-learn implementa diversos parâmetros para cada algoritmo e, em geral, possui um valor default para todos eles. Mesmo assim, podemos alterar esses valores default usando o padrão python ```param=valor```:


```
algoritmo = Algoritmo(parametro1=valor1, parametro2=valor2, ...)
```

Cada algoritmo possui uma série de parâmetros que em geral diferem uns dos outros. Para verificar quais parâmetros são implementados por cada algoritmo e o significado de cada um, podemos consultar o link http://scikit-learn.org/stable/modules/classes.html

#### 3.1 - Decision Trees

Agora vamos treinar o nosso primeiro modelo usando scikit-learn: uma árvore de decisão (decision tree). Este modelo é o padrão dos métodos baseados em informação.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dectree = DecisionTreeClassifier(criterion="gini", 
                                 splitter="best", 
                                 max_depth=None, 
                                 min_samples_split=2, 
                                 min_samples_leaf=1, 
                                 min_weight_fraction_leaf=0.0, 
                                 max_features=None, 
                                 random_state=None, 
                                 max_leaf_nodes=None, 
                                 min_impurity_decrease=0.0, 
                                 min_impurity_split=None, 
                                 class_weight=None, 
                                 presort=False)

plot_resultado(X, y, dectree, "Árvore de decisão")

### 3.2 - k-nearest neighbours

Agora que já observamos nosso primeiro algoritmo baseado em informação, vamos passar aos algoritmos baseados em similaridade. O padrão para estes algoritmos é o K-nearest neighbours. Este algoritmo mede a distância entre os pontos no feature space e utiliza os K datapoints mais próximos para determinar qual a classificação. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5, 
                           weights="uniform", 
                           algorithm="auto", 
                           leaf_size=30, 
                           p=2, 
                           metric="minkowski", 
                           metric_params=None, 
                           n_jobs=1)

plot_resultado(X, y, knn, "K-Nearest Neighbours")

### 3.3 - Naïve Bayes

O próximo algoritmo que vamos analisar é o Naïve Bayes, padrão dos métodos baseados em probabilidade.

In [ ]:
from sklearn.naive_bayes import GaussianNB
nbayes = GaussianNB(priors=None)
plot_resultado(X, y, nbayes, "Naïve Bayes")

### 3.4 - Regressão Logística

O último tipo de algoritmo que nos falta estudar é o baseado em erro. O método padrão é a Regressão Logística, similar ao método usado no ramo da estatística, com algumas alterações apenas na forma de se treinar o modelo.

In [ ]:
from sklearn.linear_model import LogisticRegression

linear = LogisticRegression(penalty="l2", 
                            dual=False, 
                            tol=0.0001, 
                            C=1.0, 
                            fit_intercept=True, 
                            intercept_scaling=1, 
                            class_weight=None, 
                            random_state=None, 
                            solver="liblinear", 
                            max_iter=100, 
                            multi_class="ovr", 
                            verbose=0, 
                            warm_start=False, 
                            n_jobs=1)

plot_resultado(X, y, linear, "Regressão Logística")

### 4 - Exercícios

#### 4.1 - Testando os hyperparâmetros

Volte aos exercícios acima e troque os hyperparâmetros *default* de cada algoritmo. Observe como o gráfico muda (ou não). O controle dos hyperparâmetros é um dos fatores que pode determinar qual o melhor algoritmo para cada caso. Na próxima lição nós vamos estudar um método de busca para otimizar a escolha dos hyperparâmetros.

Para auxiliar na tarefa de escolha, veja a documentação de cada classe nos links disponíveis em cada seção.

*Observação*: o único algoritmo que não possui hyperparâmetros é o baseado em probabilidades.

#### 4.2 - Outros algoritmos

Acesse o link abaixo e escolha outro(s) algoritmo(s) que ainda não foram vistos e treine um ou mais modelos com ele(s). Observe como o algoritmo se comporta.

Link: http://scikit-learn.org/stable/modules/classes.html

Você pode utilizar o código abaixo trocando a palavra-chave nome_do_algoritmo pelo escolhido. Não esqueça de verificar o nome da classe utilizada no ```import```!


In [ ]:
from sklearn.classe_do_algoritmo import nome_do_algoritmo
classifier = nome_do_algoritmo()
plot_resultado(X, y, classifier, "Teste")